**What it is:**  
  - This script processes a set of real estate product data to classify and enrich product attributes, then uploads and merges the cleaned data into a Snowflake database table.

**What it does:**  
  - Defines product records with basic metadata (product code, name, vehicle category).  
  - Applies classification logic to assign share classes (e.g., Institutional, Retail) and vehicle types/categories (e.g., Mutual Fund, Separate Account).  
  - Adjusts product details for ETFs by updating names and classifications accordingly.  
  - Adds mandatory static fields such as asset class, marketing flag, and performance accounts.  
  - Normalizes column names for compatibility with Snowflake.  
  - Connects to Snowflake using environment variables for credentials.  
  - Uploads the processed DataFrame to a staging table in Snowflake.  
  - Merges the staging data into the main ProductMaster table, inserting new records while avoiding duplicates.

**Why it’s important:**  
  - Ensures consistent and accurate classification of product data, which is critical for reporting, analysis, and compliance.  
  - Automates data ingestion and synchronization with the Snowflake data warehouse, improving data reliability and timeliness.  
  - Supports downstream analytics and portfolio management by maintaining up-to-date product metadata aligned with business definitions.


# **Product Master Table**

## Share Class Classification dictionary

| Label           | Meaning                                                                                         |
| --------------- | ----------------------------------------------------------------------------------------------- |
| `Institutional` | Offered through **separate accounts**, **pensions**, or direct to large institutional clients   |
| `Retail`        | Publicly available to **individual investors** (e.g. mutual funds, 401(k) plans, ETFs)          |
| `Preferred`     | Strategy targeting **preferred equity instruments**, typically REITs with fixed yields          |
| `Class A`       | Front-load share class with **upfront sales charges** and lower expense ratios                  |
| `Class B`       | **Deferred sales charge** (back-end load); often converts to Class A after a set period         |
| `Class C`       | **Level-load** class with higher annual fees, but no front- or back-end sales charges           |
| `N/A`           | No identifiable share class structure (e.g., internal models, simulated products)               |



## reit_shareclass_classifier.py

In [ ]:
import pandas as pd

# Sample ProductMaster Data
product_df = pd.DataFrame({
    "PRODUCTCODE": ["REITProd001", "REITProd002"],
    "PRODUCTNAME": [
        "Assette Real Estate Growth Fund",
        "Assette Real Estate Value Fund"
    ],
    "VEHICLECATEGORY": [
        "Mutual Fund", "ETF"
    ]
})

def assign_share_class(row):
    """
    Assigns a share class label to a product row based on keywords in the product name
    and vehicle category.

    Args:
        row (pd.Series): A row from the product DataFrame.

    Returns:
        str: The assigned share class, or 'N/A' if no match.
    """
    name = str(row.get("PRODUCTNAME", "")).lower()
    vehicle = str(row.get("VEHICLECATEGORY", "")).lower()

    if "class a" in name:
        return "Class A"
    if "class b" in name:
        return "Class B"
    if "class c" in name:
        return "Class C"
    if "preferred" in name:
        return "Preferred"
    if "institutional" in name or "separate account" in vehicle:
        return "Institutional"
    if "retail" in name or "mutual" in vehicle:
        return "Retail"
    return "N/A"

def assign_vehicle_type(row):
    """
    Determines the vehicle type of a product based on product name and vehicle category.
    Assette-approved values only.

    Args:
        row (pd.Series): A row from the product DataFrame.

    Returns:
        str: Vehicle type such as 'Separate Account', 'Mutual Fund', or 'Commingled Fund'.
    """
    name = str(row.get("PRODUCTNAME", "")).lower()
    category = str(row.get("VEHICLECATEGORY", "")).lower()

    if "separate" in category or "separate account" in name:
        return "Separate Account"
    if "mutual" in name or "mutual" in category:
        return "Mutual Fund"
    if "fund" in name and "class" in name:
        return "Commingled Fund"  # heuristic fallback for pooled vehicles
    return "Mutual Fund"  # default fallback (ETF not allowed)

def assign_vehicle_category(row):
    """
    Assigns a vehicle category value based on vehicle type.
    Separate Accounts are considered 'Segregated', all others 'Pooled'.

    Args:
        row (pd.Series): A row from the product DataFrame.

    Returns:
        str: 'Segregated' or 'Pooled'
    """
    vt = row.get("VEHICLETYPE", "")
    if vt == "Separate Account":
        return "Segregated"
    return "Pooled"

# Apply share class, vehicle type, and category logic to the DataFrame
product_df["SHARECLASS"] = product_df.apply(assign_share_class, axis=1)
product_df["VEHICLETYPE"] = product_df.apply(assign_vehicle_type, axis=1)
product_df["VEHICLECATEGORY"] = product_df.apply(assign_vehicle_category, axis=1)

def recast_etf_products(row):
    """
    For products identified as ETFs, updates the product metadata to model
    them as 'Global Real Estate Growth Fund' with institutional share class.

    Args:
        row (pd.Series): A row from the product DataFrame.

    Returns:
        pd.Series: The updated row with adjusted product fields.
    """
    name = str(row.get("PRODUCTNAME", "")).lower()
    vehicle = str(row.get("VEHICLECATEGORY", "")).lower()

    if "etf" in name or "etf" in vehicle:
        # Update product name to represent modeled product
        row["PRODUCTNAME"] = f"Global Real Estate Growth Fund ({row['PRODUCTCODE']})"
        row["VEHICLETYPE"] = "Commingled Fund"
        row["VEHICLECATEGORY"] = "Pooled"
        # Default to institutional share class for ETFs
        row["SHARECLASS"] = "Institutional"

    return row

# Apply ETF recasting logic
product_df = product_df.apply(recast_etf_products, axis=1)

# Fill remaining mandatory fields for testing/demo purposes
product_df["ASSETCLASS"] = "Equity"
product_df["ISMARKETED"] = 1
product_df["PARENTPRODUCTCODE"] = "PRDK_MASTER"
product_df["PERFORMANCEACCOUNT"] = product_df["PRODUCTCODE"].apply(lambda x: f"{x}_PERF")
product_df["REPRESENTATIVEACCOUNT"] = product_df["PRODUCTCODE"].apply(lambda x: f"{x}_REP")
product_df["STRATEGY"] = "REIT"

# Print the final product DataFrame with key columns
print(product_df[[
    "PRODUCTCODE", "PRODUCTNAME", "ASSETCLASS", "ISMARKETED", "PARENTPRODUCTCODE",
    "PERFORMANCEACCOUNT", "REPRESENTATIVEACCOUNT", "SHARECLASS",
    "STRATEGY", "VEHICLECATEGORY", "VEHICLETYPE"
]])


   PRODUCTCODE                      PRODUCTNAME ASSETCLASS  ISMARKETED  \
0  REITProd001  Assette Real Estate Growth Fund     Equity           1   
1  REITProd002   Assette Real Estate Value Fund     Equity           1   

  PARENTPRODUCTCODE PERFORMANCEACCOUNT REPRESENTATIVEACCOUNT SHARECLASS  \
0       PRDK_MASTER   REITProd001_PERF       REITProd001_REP     Retail   
1       PRDK_MASTER   REITProd002_PERF       REITProd002_REP        N/A   

  STRATEGY VEHICLECATEGORY  VEHICLETYPE  
0     REIT          Pooled  Mutual Fund  
1     REIT          Pooled  Mutual Fund  


How it works:

- The logic assumes a seller perspective, classifying investment products based on how they're positioned in name and vehicle type — not ticker identity.

- ETF-based products are no longer treated literally. If the product name or vehicle category contains "ETF":

    - The product is recast as a modeled strategy (e.g., “Global Real Estate Growth Fund”)

    - VEHICLETYPE is set to "Commingled Fund"

    - VEHICLECATEGORY is set to "Pooled"

    - SHARECLASS defaults to "Institutional"

- The code then uses keyword heuristics to assign a SHARECLASS label:

| Tag             | Trigger Logic                                                              |
| --------------- | -------------------------------------------------------------------------- |
| `Class A/B/C`   | Product name contains `"class a"`, `"class b"`, or `"class c"`             |
| `Preferred`     | Product name contains `"preferred"`                                        |
| `Institutional` | Product name or vehicle mentions `"institutional"` or `"separate account"` |
| `Retail`        | Name or vehicle contains `"retail"` or `"mutual"`                          |
| `N/A`           | No known indicator present                                                 |


This approach focuses on fund packaging logic, not holdings.

Further use / Pipeline integration:

- Replace the sample product_df with one read from the Snowflake table or staging DataFrame

- Return this DataFrame directly to be written to staging via write_pandas() or passed to the next ETL step

## shareclass_validation.py

In [ ]:
import pandas as pd
import logging

# Set up logging configuration to capture validation info and errors
logging.basicConfig(
    filename="shareclass_validation.log",
    filemode="a",
    format="%(asctime)s | %(levelname)s | %(message)s",
    level=logging.INFO
)

# Define valid share class labels for validation checks
VALID_CLASSES = {
    "Preferred", "Institutional", "Retail",
    "Class A", "Class B", "Class C", "N/A"
}

def classify_shareclass_fund(df: pd.DataFrame) -> pd.DataFrame:
    """
    Assigns the SHARECLASS column for each product in the DataFrame
    based on keywords found in PRODUCTNAME and VEHICLECATEGORY fields.

    Args:
        df (pd.DataFrame): Input product DataFrame.

    Returns:
        pd.DataFrame: DataFrame with SHARECLASS column assigned.
    """
    def assign(row):
        name = str(row.get("PRODUCTNAME", "")).lower()
        vehicle = str(row.get("VEHICLECATEGORY", "")).lower()

        if "class a" in name:
            return "Class A"
        if "class b" in name:
            return "Class B"
        if "class c" in name:
            return "Class C"
        if "preferred" in name:
            return "Preferred"
        if "institutional" in name or "separate account" in vehicle or "institutional" in vehicle:
            return "Institutional"
        if "retail" in name or "mutual" in vehicle:
            return "Retail"
        return "N/A"

    df["SHARECLASS"] = df.apply(assign, axis=1)
    return df

def validate_shareclass(df: pd.DataFrame):
    """
    Validates the SHARECLASS values in the DataFrame against the valid classes.
    Logs summary, warnings for missing values, and errors for invalid values.
    Raises an exception if invalid SHARECLASS values exceed 10% of total.

    Args:
        df (pd.DataFrame): Input DataFrame with SHARECLASS column.
    """
    total = len(df)
    missing = df["SHARECLASS"].isna().sum()
    invalid = df[~df["SHARECLASS"].isin(VALID_CLASSES)]

    logging.info(f"Validating SHARECLASS on {total} products.")

    if missing > 0:
        logging.warning(f"{missing} rows missing SHARECLASS values.")

    if not invalid.empty:
        logging.error(f"{len(invalid)} invalid SHARECLASS values found:")
        for _, row in invalid.iterrows():
            logging.error(f"PRODUCTCODE={row.get('PRODUCTCODE')} | SHARECLASS='{row.get('SHARECLASS')}'")

    if total > 0 and len(invalid) / total > 0.1:
        raise ValueError("More than 10% of SHARECLASS values are invalid.")

    logging.info("SHARECLASS validation complete.")

def preview_shareclass_issues(df: pd.DataFrame):
    """
    Prints a preview of rows with missing or invalid SHARECLASS values.

    Args:
        df (pd.DataFrame): DataFrame with SHARECLASS column.
    """
    bad = df[df["SHARECLASS"].isna() | ~df["SHARECLASS"].isin(VALID_CLASSES)]
    if bad.empty:
        print("All SHARECLASS values are valid.")
    else:
        print("Issues found in SHARECLASS:")
        print(bad[["PRODUCTCODE", "PRODUCTNAME", "SHARECLASS"]])


# **Connecting to Snowflake**

## Establishing Connection

You must download the env.txt file which is located in my GDrive folder ("Shiv-Code") and upload using the "Choose Files" widget after running the below cell.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving env.txt to env.txt


In [ ]:
import os

if os.path.exists("env.txt"):
    os.rename("env.txt", "env")
    print("Renamed env.txt to env")
else:
    print("File not found. Make sure you uploaded env.txt.")

Renamed env.txt to env


In [ ]:
!pip install snowflake-connector-python python-dotenv

In [ ]:
from dotenv import load_dotenv
load_dotenv("env")  # This loads the Snowflake credentials into the environment

True

In [ ]:
import snowflake.connector
import os

# Pull in credentials
conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    role=os.getenv("SNOWFLAKE_ROLE"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA")
)

# Run a simple test query
cur = conn.cursor()
cur.execute("SELECT CURRENT_USER(), CURRENT_ROLE(), CURRENT_DATABASE();")
for row in cur:
    print(row)

cur.close()
conn.close()

('SHIVNEETN', 'AST_REALESTATE_DB_RW', 'AST_REALESTATE_DB')


## Sample code for updating 'ShareClass' column from functional code

In [ ]:
import pandas as pd
import os
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

# Sample ProductMaster Data
product_df = pd.DataFrame({
    "PRODUCTCODE": ["REITProd001", "REITProd002"],
    "PRODUCTNAME": [
        "Assette Real Estate Growth Fund",
        "Assette Real Estate Value Fund"
    ],
    "VEHICLECATEGORY": [
        "Mutual Fund", "ETF"
    ]
})

def assign_share_class(row):
    """
    Assign SHARECLASS based on keywords in PRODUCTNAME and VEHICLECATEGORY.

    Args:
        row (pd.Series): Row of product data.

    Returns:
        str: Assigned share class label.
    """
    name = str(row.get("PRODUCTNAME", "")).lower()
    vehicle = str(row.get("VEHICLECATEGORY", "")).lower()

    if "class a" in name:
        return "Class A"
    if "class b" in name:
        return "Class B"
    if "class c" in name:
        return "Class C"
    if "preferred" in name:
        return "Preferred"
    if "institutional" in name or "separate account" in vehicle:
        return "Institutional"
    if "retail" in name or "mutual" in vehicle:
        return "Retail"
    return "N/A"

def assign_vehicle_type(row):
    """
    Determine VEHICLETYPE based on PRODUCTNAME and VEHICLECATEGORY.

    Args:
        row (pd.Series): Row of product data.

    Returns:
        str: Vehicle type label.
    """
    name = str(row.get("PRODUCTNAME", "")).lower()
    category = str(row.get("VEHICLECATEGORY", "")).lower()

    if "separate" in category or "separate account" in name:
        return "Separate Account"
    if "mutual" in name or "mutual" in category:
        return "Mutual Fund"
    if "fund" in name and "class" in name:
        return "Commingled Fund"
    return "Mutual Fund"

def assign_vehicle_category(row):
    """
    Assign VEHICLECATEGORY based on VEHICLETYPE.

    Args:
        row (pd.Series): Row of product data.

    Returns:
        str: Vehicle category label ("Segregated" or "Pooled").
    """
    vt = row.get("VEHICLETYPE", "")
    if vt == "Separate Account":
        return "Segregated"
    return "Pooled"

def recast_etf_products(row):
    """
    Adjust product details for ETF products.

    Args:
        row (pd.Series): Row of product data.

    Returns:
        pd.Series: Modified row with updated ETF fields.
    """
    name = str(row.get("PRODUCTNAME", "")).lower()
    vehicle = str(row.get("VEHICLECATEGORY", "")).lower()

    if "etf" in name or "etf" in vehicle:
        row["PRODUCTNAME"] = f"Global Real Estate Growth Fund ({row['PRODUCTCODE']})"
        row["VEHICLETYPE"] = "Commingled Fund"
        row["VEHICLECATEGORY"] = "Pooled"
        row["SHARECLASS"] = "Institutional"
    return row

# Apply classification logic to DataFrame
product_df["SHARECLASS"] = product_df.apply(assign_share_class, axis=1)
product_df["VEHICLETYPE"] = product_df.apply(assign_vehicle_type, axis=1)
product_df["VEHICLECATEGORY"] = product_df.apply(assign_vehicle_category, axis=1)
product_df = product_df.apply(recast_etf_products, axis=1)

# Add remaining required columns with static/default values
product_df["ASSETCLASS"] = "Equity"
product_df["ISMARKETED"] = 1
product_df["PARENTPRODUCTCODE"] = "PRDK_MASTER"
product_df["PERFORMANCEACCOUNT"] = product_df["PRODUCTCODE"].apply(lambda x: f"{x}_PERF")
product_df["REPRESENTATIVEACCOUNT"] = product_df["PRODUCTCODE"].apply(lambda x: f"{x}_REP")
product_df["STRATEGY"] = "REIT"

# Normalize column names to uppercase for Snowflake compatibility
product_df.columns = product_df.columns.str.upper()

# Connect to Snowflake using environment variables for credentials
conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    role=os.getenv("SNOWFLAKE_ROLE"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA")
)

with conn.cursor() as cur:
    # Set database and schema explicitly
    cur.execute("USE DATABASE AST_REALESTATE_DB")
    cur.execute("USE SCHEMA DBO")

    # Upload DataFrame to Snowflake staging table (overwrite if exists)
    success, nchunks, nrows, _ = write_pandas(conn, product_df, "STG_PRODUCTMASTER", overwrite=True)
    print(f"Uploaded {nrows} rows to STG_PRODUCTMASTER")

    # Merge new rows into ProductMaster table, insert only if PRODUCTCODE not already present
    cur.execute("""
        MERGE INTO ProductMaster AS target
        USING STG_PRODUCTMASTER AS source
        ON target.PRODUCTCODE = source.PRODUCTCODE
        WHEN NOT MATCHED THEN
        INSERT (
            PRODUCTCODE,
            PRODUCTNAME,
            ASSETCLASS,
            ISMARKETED,
            PARENTPRODUCTCODE,
            PERFORMANCEACCOUNT,
            REPRESENTATIVEACCOUNT,
            SHARECLASS,
            STRATEGY,
            VEHICLECATEGORY,
            VEHICLETYPE
        )
        VALUES (
            source.PRODUCTCODE,
            source.PRODUCTNAME,
            source.ASSETCLASS,
            source.ISMARKETED,
            source.PARENTPRODUCTCODE,
            source.PERFORMANCEACCOUNT,
            source.REPRESENTATIVEACCOUNT,
            source.SHARECLASS,
            source.STRATEGY,
            source.VEHICLECATEGORY,
            source.VEHICLETYPE
        )
    """)
    print("New rows successfully inserted into ProductMaster")


Uploaded 2 rows to STG_PRODUCTMASTER
New rows successfully inserted into ProductMaster


# **Data Flow & Architecture Mapping (WIP)**

## Source to Storage Mapping

| Stage                          | Description                                                                                          |
| ------------------------------ | ---------------------------------------------------------------------------------------------------- |
| **Source**                     | Synthetic or internal product metadata (`PRODUCTCODE`, `PRODUCTNAME`, `VEHICLECATEGORY`)             |
| **Extraction**                 | Loaded as `product_df` into Python from Snowflake, Excel seed file, or local seed JSON               |
| **Transformation**             | Classification logic applies business rules:                                                         |
|  • `assign_share_class()`      | Infers SHARECLASS from `PRODUCTNAME` and `VEHICLECATEGORY`                                           |
|  • `assign_vehicle_type()`     | Maps to Assette-approved VEHICLETYPE values                                                          |
|  • `assign_vehicle_category()` | Maps VEHICLECATEGORY to `Segregated` or `Pooled`                                                     |
|  • `recast_etf_products()`     | Rewrites ETF-flagged products as modeled commingled institutional vehicles                           |
| **Validation**                 | `validate_shareclass()` checks for invalid or missing SHARECLASS values; logs issues for diagnostics |
| **Storage**                    | Final enriched `product_df` is directly loaded to Snowflake `PRODUCTMASTER` via `write_pandas()`     |
| **Consumption**                | Queried by Assette UI, fact sheet generator, classification reports, or internal audit pipelines     |


## Data Flow Diagram

          +---------------------+
          | Raw Input (CSV/DB)  |  ← manual seed or staging
          +---------------------+
                     |
         [read_csv or Snowflake SELECT]
                     |
                     v
          +-----------------------------+
          |   recast_fund_products()     |  ← Normalize Fund records
          +-----------------------------+
                     |
                     v
          +-----------------------------+
          | assign_share_class()        |  ← Infer share class
          | assign_vehicle_type()       |  ← Map vehicle type
          | assign_vehicle_category()   |  ← Pooled or segregated
          +-----------------------------+
                     |
                     v
          +-----------------------------+
          | validate_shareclass()       |  ← Error handling / QA
          +-----------------------------+
                     |
                     v
          +-----------------------------+
          | write_pandas() to Snowflake |
          | (PRODUCTMASTER table)       |
          +-----------------------------+
                     |
                     v
          +-----------------------------+
          | Access via Assette UI, SQL  |
          +-----------------------------+


## Transformation Logic Documentation

SHARECLASS:

| Transformation  | Description                                                          |
| --------------- | -------------------------------------------------------------------- |
| Class Detection | Tags based on `"Class A"`, `"Class B"`, `"Class C"` in `PRODUCTNAME` |
| Preferred       | Detected from presence of `"Preferred"` keyword                      |
| Institutional   | Matches `"Institutional"` or `"Separate Account"` in name or vehicle |
| Retail          | Matches `"Retail"` or `"Mutual"` in name or vehicle                  |
| ETF Recast      | Defaulted to `"Institutional"` if recast as ETF-modeled strategy     |
| Fallback        | Defaults to `"N/A"` if no trigger matched                            |
| Data Type       | `VARCHAR` (non-nullable in final table)                              |



VEHICLETYPE:

| Transformation   | Description                                                              |
| ---------------- | ------------------------------------------------------------------------ |
| Separate Account | Detected from `"Separate Account"` in `VEHICLECATEGORY` or `PRODUCTNAME` |
| Mutual Fund      | Default fallback if `"Mutual"` found or other logic doesn’t apply        |
| Commingled Fund  | Assigned when ETF-based product is recast                                |
| Data Type        | `VARCHAR` – uses Assette's approved enumeration list                     |



VEHICLECATEGORY:

| Transformation | Description                                     |
| -------------- | ----------------------------------------------- |
| Segregated     | Assigned if `VEHICLETYPE == "Separate Account"` |
| Pooled         | Assigned for all other vehicle types            |
| Data Type      | `VARCHAR`, maps to Assette display terminology  |



ETF-Based Recast Logic:

| Trigger                                           | Transformation                                                             |
| ------------------------------------------------- | -------------------------------------------------------------------------- |
| `PRODUCTNAME` or `VEHICLECATEGORY` contains "ETF" | Recast as modeled strategy                                                 |
| PRODUCTNAME updated                               | Formatted as `"Global Real Estate Growth Fund (PRODUCTCODE)"`              |
| VEHICLETYPE                                       | Set to `"Commingled Fund"`                                                 |
| VEHICLECATEGORY                                   | Set to `"Pooled"`                                                          |
| SHARECLASS                                        | Defaulted to `"Institutional"`                                             |
| Purpose                                           | Ensure ETF products are modeled as fund wrappers with appropriate metadata |


## Future Enhancements

| Feature                  | Description                                                               |
| ------------------------ | ------------------------------------------------------------------------- |
| REIT Exposure Join       | Join to HoldingsDetails; if ≥75% REITs, tag strategy or override metadata |
| Audit Fields             | Add `INGESTED_TIMESTAMP`, `CLASSIFICATION_METHOD`, and `SOURCE` columns   |
| Override Table           | Maintain a small lookup for manual corrections or edge-case tickers       |
| Business Rules as Config | Extract logic into YAML or JSON for maintainability and UI control        |
| CI/CD Integration        | Deploy logic into dbt or orchestration pipeline with scheduled validation |
